# $n$-Local Chain Depolarizing Noise Visibility

This notebook verifies the reported visibility for depolarizing noise on both entanglement sources and qubit communication.

In [1]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import time

from context import qnetvo as QNopt

### Two-Qubit Depolarizing Channel

Two-qubit depolarizing noise is applied to each entanglement source. We apply this noise with a custom two-qubit depolarizing channel.

In the $n$-local chain literature, the depolarizing channel is typically expressed as 

$$
    \mathcal{D}(\rho) = v \rho + (1-v)\frac{1}{d}\mathbb{I} 
$$

where $d$ is the Hilbert space dimension and $v\in[-\frac{1}{d^2-1},1]$ is referred to the visibility.
Most references lower bound the visibility at $v=0$, however, $v=\frac{1}{d^2-1}$ yields a completely positive channel and is therefore physical.

PennyLane uses kraus operators to represent a quantum depolarizing channel as

$$
K_{0,0} = \sqrt{(1-p)}\mathbb{I}, \quad K_{j,k} = \sqrt{\frac{p}{d^2-1}}\sigma_j\otimes\sigma_k
$$

where $\vec{\sigma} = \{\mathbb{I}, \sigma_x, \sigma_y, \sigma_z \}$ and $j$ and $k$ are considered over all values except $j=k=0$.
In this channel representation, $p\in[0,1]$.

To convert the visibility $v$ to $p$ we use $(1-p) = (1 + 15v)/16$ or $p = (1-v)15/16$.
By this rescaling, we reproduce the known depolarizing visibility of CHSH, bilocal, trilocal, and quadlocal scenarios.

In [2]:
from pennylane.operation import Channel

class TwoQubitDepolarizing(Channel):

    num_params = 1
    num_wires = 2
    par_domain = "R"
    grad_method = "A"
    grad_recipe = ([[1, 0, 1], [-1, 0, 0]],)

    @classmethod
    def _kraus_matrices(cls, *params):
        p = params[0]
        
        paulis = [
            np.eye(2),
            np.array([[0, 1], [1, 0]]),
            np.array([[0, -1j], [1j, 0]]),
            np.array([[1, 0],[0, -1]])
        ]
        
        kraus_ops = []
        for i in range(4):
            for j in range(4):
                kraus_ops.append(np.kron(paulis[i],paulis[j]))
        
        kraus_ops[0] = kraus_ops[0] * np.sqrt(1 - p)
        for i in range(1,16):
            kraus_ops[i] = kraus_ops[i] * np.sqrt(p / 15)
        
        return kraus_ops
    
# updating default.mixed device to have knowledge of TwoQubitDepolarizing Channel.
dev = qml.device("default.mixed", wires=[0,1]) 
dev.operations.update(["TwoQubitDepolarizing"])

In [3]:
def bell_state_local_RY(settings, wires):
    qml.Hadamard(wires[0])
    qml.CNOT(wires=wires[0:2])
        
    qml.RY(settings[0], wires=wires[0])
    qml.RY(settings[1], wires=wires[1])
    
def arbitrary_prepare_nodes(n):
    return [QNopt.PrepareNode(1, [2*i, 2*i + 1], qml.templates.subroutines.ArbitraryUnitary, 15) for i in range(n)]

def arbitrary_measure_nodes(n):
    meas_nodes = [QNopt.MeasureNode(2, 2, [0], qml.templates.subroutines.ArbitraryUnitary, 3)]
    meas_nodes.extend([QNopt.MeasureNode(2,2, [2*i + 1, 2*i + 2], qml.templates.subroutines.ArbitraryUnitary, 15) for i in range(0,n-1)])
    meas_nodes.append(QNopt.MeasureNode(2, 2, [2*n-1], qml.templates.subroutines.ArbitraryUnitary, 3))

    return meas_nodes

def static_nlocal_prepare_nodes(n):
    return [QNopt.PrepareNode(1, [2*i, 2*i + 1], bell_state_local_RY, 2) for i in range(n)]

def local_RY_measure_nodes(n):
    meas_nodes = [QNopt.MeasureNode(2, 2, [0], QNopt.local_RY, 1)]
    meas_nodes.extend([
        QNopt.MeasureNode(2, 2, [2*i + 1, 2*i + 2], QNopt.local_RY, 2) for i in range(0,n-1)
    ])
    meas_nodes.append(QNopt.MeasureNode(2, 2, [2*n-1], QNopt.local_RY, 1))
    
    return meas_nodes

def qubit_noise_nodes(n, fn, chan_params):
    return [QNopt.NoiseNode([i], lambda settings, wires: fn(*chan_params[i], wires=wires[0])) for i in range(2*n)]

def entanglement_noise_nodes(n, fn, chan_params):
    return [QNopt.NoiseNode([2*i, 2*i + 1], lambda settings, wires: fn(*chan_params[i], wires=wires[0:2])) for i in range(n)]

## CHSH Visibility



In [4]:
n = 1
v1 = 1/2**(1/4)  # qubit depolarizing
v2 = 1/2**(1/2)  # entangled source depolarizing

p1 = (1-v1)*3/4
p2 = (1-v2)*15/16

arb_chsh_prep_nodes = arbitrary_prepare_nodes(n)
bell_prep_nodes = static_nlocal_prepare_nodes(n)

RY_chsh_meas_nodes = [
    QNopt.MeasureNode(2,2,[0],QNopt.local_RY, 1),
    QNopt.MeasureNode(2,2,[1],QNopt.local_RY, 1),
]
arb_chsh_meas_nodes = [
    QNopt.MeasureNode(2,2,[0],qml.templates.subroutines.ArbitraryUnitary, 3),
    QNopt.MeasureNode(2,2,[1],qml.templates.subroutines.ArbitraryUnitary, 3),
]

depolarized_qubit_nodes = qubit_noise_nodes(n, qml.DepolarizingChannel, [[p1]]*2)
depolarized_prep_nodes = entanglement_noise_nodes(n, TwoQubitDepolarizing, [[p2]])

In [5]:
%%time

depolarized_qubit_chsh_ansatz = QNopt.NetworkAnsatz(bell_prep_nodes, RY_chsh_meas_nodes, depolarized_qubit_nodes)
depolarized_qubit_chsh_cost = QNopt.chsh_inequality_cost(depolarized_qubit_chsh_ansatz)

np.random.seed(1)
depolarized_qubit_chsh_settings = depolarized_qubit_chsh_ansatz.rand_scenario_settings()

depolarized_qubit_chsh_opt_dict = QNopt.gradient_descent(
    depolarized_qubit_chsh_cost,
    depolarized_qubit_chsh_settings,
    num_steps=18,
    step_size=0.2,
    sample_width = 6
)

print("max score : ", depolarized_qubit_chsh_opt_dict["opt_score"])

iteration :  0 , score :  -1.3978435300167364
elapsed time :  0.043894052505493164
iteration :  6 , score :  1.849507281053557
elapsed time :  0.0375826358795166
iteration :  12 , score :  1.9883087659793186
elapsed time :  0.11723613739013672
max score :  1.9991885381542227
CPU times: user 759 ms, sys: 26.7 ms, total: 786 ms
Wall time: 769 ms


In [6]:
%%time

depolarized_prep_chsh_ansatz = QNopt.NetworkAnsatz(bell_prep_nodes, RY_chsh_meas_nodes, depolarized_prep_nodes)
depolarized_prep_chsh_cost = QNopt.chsh_inequality_cost(depolarized_prep_chsh_ansatz)

np.random.seed(1)
depolarized_prep_chsh_settings = depolarized_prep_chsh_ansatz.rand_scenario_settings()

depolarized_prep_chsh_opt_dict = QNopt.gradient_descent(
    depolarized_prep_chsh_cost,
    depolarized_prep_chsh_settings,
    num_steps=18,
    step_size=0.2,
    sample_width = 6
)

print("max score : ", depolarized_prep_chsh_opt_dict["opt_score"])

iteration :  0 , score :  -1.3978435300167371
elapsed time :  0.08796525001525879
iteration :  6 , score :  1.8495072810535576
elapsed time :  0.08509278297424316
iteration :  12 , score :  1.9883087659793182
elapsed time :  0.08362698554992676
max score :  1.9991885381542223
CPU times: user 1.51 s, sys: 22 ms, total: 1.53 s
Wall time: 1.52 s


In [7]:
%%time

arb_depolarized_qubit_chsh_ansatz = QNopt.NetworkAnsatz(arb_chsh_prep_nodes, arb_chsh_meas_nodes, depolarized_qubit_nodes)
arb_depolarized_qubit_chsh_cost = QNopt.chsh_inequality_cost(arb_depolarized_qubit_chsh_ansatz)

np.random.seed(1)
arb_depolarized_qubit_chsh_settings = arb_depolarized_qubit_chsh_ansatz.rand_scenario_settings()

arb_depolarized_qubit_chsh_opt_dict = QNopt.gradient_descent(
    arb_depolarized_qubit_chsh_cost,
    arb_depolarized_qubit_chsh_settings,
    num_steps=18,
    step_size=0.25,
    sample_width = 6
)

print("max score : ", arb_depolarized_qubit_chsh_opt_dict["opt_score"])

iteration :  0 , score :  -0.4043861425360129
elapsed time :  0.7223312854766846
iteration :  6 , score :  1.8848667392922351
elapsed time :  0.6924920082092285
iteration :  12 , score :  1.982369366669074
elapsed time :  0.7062621116638184
max score :  1.9964198923138066
CPU times: user 11.8 s, sys: 192 ms, total: 12 s
Wall time: 12 s


In [8]:
%%time

arb_depolarized_prep_chsh_ansatz = QNopt.NetworkAnsatz(arb_chsh_prep_nodes, arb_chsh_meas_nodes, depolarized_prep_nodes)
arb_depolarized_prep_chsh_cost = QNopt.chsh_inequality_cost(arb_depolarized_prep_chsh_ansatz)

np.random.seed(1)
arb_depolarized_prep_chsh_settings = arb_depolarized_prep_chsh_ansatz.rand_scenario_settings()

arb_depolarized_prep_chsh_opt_dict = QNopt.gradient_descent(
    arb_depolarized_prep_chsh_cost,
    arb_depolarized_prep_chsh_settings,
    num_steps=18,
    step_size=0.25,
    sample_width = 6
)

print("max score : ", arb_depolarized_prep_chsh_opt_dict["opt_score"])

iteration :  0 , score :  -0.40438614253601335
elapsed time :  0.7887029647827148
iteration :  6 , score :  1.8848667392922334
elapsed time :  0.9663879871368408
iteration :  12 , score :  1.9823693666690743
elapsed time :  0.8200299739837646
max score :  1.9964198923138063
CPU times: user 16.6 s, sys: 135 ms, total: 16.7 s
Wall time: 16.8 s


## Bilocal Chain Visibility

In [18]:
n = 2
v1 = 1/2**(1/4)
v2 = 1/2**(1/2)

p1 = (1 - v1)*3/4
p2 = (1 - v2)*15/16

arb_prep_nodes = arbitrary_prepare_nodes(n)
bell_prep_nodes = static_nlocal_prepare_nodes(n)

local_meas_nodes = local_RY_measure_nodes(n)
arb_meas_nodes = arbitrary_measure_nodes(n)

depolarized_qubit_nodes = qubit_noise_nodes(n, qml.DepolarizingChannel, [[p1]]*4)
depolarized_prep_nodes = entanglement_noise_nodes(n, TwoQubitDepolarizing, [[p2]]*2)

In [19]:
%%time

depolarized_qubit_bilocal_ansatz = QNopt.NetworkAnsatz(bell_prep_nodes, local_meas_nodes, depolarized_qubit_nodes)
depolarized_qubit_bilocal_cost = QNopt.nlocal_chain_cost_22(depolarized_qubit_bilocal_ansatz)
noiseless_cost = QNopt.nlocal_chain_cost_22(QNopt.NetworkAnsatz(bell_prep_nodes,local_meas_nodes))

np.random.seed(1)
depolarized_qubit_bilocal_settings = depolarized_qubit_bilocal_ansatz.rand_scenario_settings()

depolarized_qubit_bilocal_opt_dict = QNopt.gradient_descent(
    depolarized_qubit_bilocal_cost,
    depolarized_qubit_bilocal_settings,
    num_steps=12,
    step_size=1.1,
    sample_width = 3
)

print("max score : ", depolarized_qubit_bilocal_opt_dict["opt_score"])
print("noiseless score : ", noiseless_cost(depolarized_qubit_bilocal_opt_dict["opt_settings"]))

iteration :  0 , score :  0.642637739143719
elapsed time :  0.3215909004211426
iteration :  3 , score :  0.9811323177379109
elapsed time :  1.1097133159637451
iteration :  6 , score :  0.9979186468786521
elapsed time :  0.3381540775299072
iteration :  9 , score :  0.9997398097084393
elapsed time :  0.3406400680541992
max score :  0.9999654026506961
noiseless score :  -1.4141646343324876
CPU times: user 4.23 s, sys: 351 ms, total: 4.58 s
Wall time: 4.65 s


In [11]:
%%time

depolarized_prep_bilocal_ansatz = QNopt.NetworkAnsatz(bell_prep_nodes, local_meas_nodes, depolarized_prep_nodes)
depolarized_prep_bilocal_cost = QNopt.nlocal_chain_cost_22(depolarized_prep_bilocal_ansatz)

np.random.seed(1)
depolarized_prep_bilocal_settings = depolarized_prep_bilocal_ansatz.rand_scenario_settings()

depolarized_prep_bilocal_opt_dict = QNopt.gradient_descent(
    depolarized_prep_bilocal_cost,
    depolarized_prep_bilocal_settings,
    num_steps=12,
    step_size=1.1,
    sample_width = 3
)

print("max score : ", depolarized_prep_bilocal_opt_dict["opt_score"])

iteration :  0 , score :  0.6426377391437188
elapsed time :  0.969027042388916
iteration :  3 , score :  0.9811323177379105
elapsed time :  0.9351539611816406
iteration :  6 , score :  0.9979186468786521
elapsed time :  0.9161720275878906
iteration :  9 , score :  0.9997398097084393
elapsed time :  0.8145499229431152
max score :  0.9999654026506961
CPU times: user 10.8 s, sys: 80.8 ms, total: 10.9 s
Wall time: 10.9 s


In [12]:
%%time

arb_depolarized_qubit_bilocal_ansatz = QNopt.NetworkAnsatz(arb_prep_nodes, arb_meas_nodes, depolarized_qubit_nodes)
arb_depolarized_qubit_bilocal_cost = QNopt.nlocal_chain_cost_22(arb_depolarized_qubit_bilocal_ansatz)

np.random.seed(1)
arb_depolarized_qubit_bilocal_settings = arb_depolarized_qubit_bilocal_ansatz.rand_scenario_settings()

arb_depolarized_qubit_bilocal_opt_dict = QNopt.gradient_descent(
    arb_depolarized_qubit_bilocal_cost,
    arb_depolarized_qubit_bilocal_settings,
    num_steps=15,
    step_size=1.1,
    sample_width = 5
)

print("max score : ", arb_depolarized_qubit_bilocal_opt_dict["opt_score"])

iteration :  0 , score :  0.184316004489942
elapsed time :  10.186198949813843
iteration :  5 , score :  0.8747908363695005
elapsed time :  9.567085027694702
iteration :  10 , score :  0.987471890280099
elapsed time :  9.39594292640686
max score :  0.9974032084261946
CPU times: user 2min 28s, sys: 637 ms, total: 2min 29s
Wall time: 2min 29s


## Trilocal Chain Depolarized State Visibility

In [13]:
n = 3
v1 = 1/2**(1/6)
v2 = 1/2**(1/3)

p1 = (1 - v1)*3/4
p2 = (1 - v2)*15/16

arb_prep_nodes = arbitrary_prepare_nodes(n)
bell_prep_nodes = static_nlocal_prepare_nodes(n)

local_meas_nodes = local_RY_measure_nodes(n)
arb_meas_nodes = arbitrary_measure_nodes(n)

depolarized_qubit_nodes = qubit_noise_nodes(n, qml.DepolarizingChannel, [[p1]]*6)
depolarized_prep_nodes = entanglement_noise_nodes(n, TwoQubitDepolarizing, [[p2]]*3)

In [14]:
%%time

depolarized_qubit_trilocal_ansatz = QNopt.NetworkAnsatz(bell_prep_nodes, local_meas_nodes, depolarized_qubit_nodes)
depolarized_qubit_trilocal_cost = QNopt.nlocal_chain_cost_22(depolarized_qubit_trilocal_ansatz)

np.random.seed(1)
depolarized_qubit_trilocal_settings = depolarized_qubit_trilocal_ansatz.rand_scenario_settings()

depolarized_qubit_trilocal_opt_dict = QNopt.gradient_descent(
    depolarized_qubit_trilocal_cost,
    depolarized_qubit_trilocal_settings,
    num_steps=18,
    step_size=1.1,
    sample_width = 3
)

print("max score : ", depolarized_qubit_bilocal_opt_dict["opt_score"])

iteration :  0 , score :  0.708473424641541
elapsed time :  3.4709599018096924
iteration :  3 , score :  0.9301591674832705
elapsed time :  3.327565908432007
iteration :  6 , score :  0.9887344661964581
elapsed time :  3.363426685333252
iteration :  9 , score :  0.9983493628879575
elapsed time :  3.5418219566345215
iteration :  12 , score :  0.9997617497218667
elapsed time :  3.3899240493774414
iteration :  15 , score :  0.9999655544878412
elapsed time :  3.289108991622925
max score :  0.9999654026506961
CPU times: user 1min 2s, sys: 368 ms, total: 1min 2s
Wall time: 1min 3s


## Quadlocal Chain Depolarizing Noise Visibility

In [15]:
n = 4

v1 = 1/2**(1/8)
v2 = 1/2**(1/4)

p1 = (1 - v1)*3/4
p2 = (1 - v2)*15/16

arb_prep_nodes = arbitrary_prepare_nodes(n)
bell_prep_nodes = static_nlocal_prepare_nodes(n)

local_meas_nodes = local_RY_measure_nodes(n)

depolarized_qubit_nodes = qubit_noise_nodes(n, qml.DepolarizingChannel, [[p1]]*8)


In [16]:
%%time

depolarized_quadlocal_ansatz = QNopt.NetworkAnsatz(bell_prep_nodes, local_meas_nodes, depolarized_qubit_nodes)
depolarized_quadlocal_cost = QNopt.nlocal_chain_cost_22(depolarized_quadlocal_ansatz)

np.random.seed(1)
depolarized_quadlocal_settings = depolarized_quadlocal_ansatz.rand_scenario_settings()

depolarized_quadlocal_opt_dict = QNopt.gradient_descent(
    depolarized_quadlocal_cost,
    depolarized_quadlocal_settings,
    num_steps=6,
    step_size=1.1,
    sample_width = 3
)

print("max score : ", depolarized_quadlocal_opt_dict["opt_score"])

iteration :  0 , score :  0.14908011051425443
elapsed time :  67.9748055934906
iteration :  3 , score :  0.9189466135624951
elapsed time :  65.96015191078186
max score :  0.9983287990261192
CPU times: user 6min 45s, sys: 695 ms, total: 6min 46s
Wall time: 6min 54s


## Conclusion

This notebook reproduces the observation that $V_{NLOC} = \frac{1}{2}$ requiring each entangled state preparation to have a visibility of $\frac{1}{2^{1/n}}$. For noise on individual qubits requires a $V_{qubit} = \frac{1}{2^{1/2n}}$. As the length of the chain increases, the required visibility on each entanglement sources trends to 0. This scaling is unforunate because as the chain gets longer, the entanglement sources need to be of a higher quality to make a violation to the $n$-local set.
